In [ ]:
import os
import json
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch_itl import model, sampler, cost, kernel, estimator

### Set paths and load model config / ckpt

In [ ]:
# Set trained model paths
base_experiment_path = './LS_Experiments'
model_name = 'Rafd_itl_model_20201112-152502'

# get model config and ckpt
base_model_path = os.path.join(base_experiment_path, model_name, 'model/')
for fname in os.listdir(base_model_path):
    if ('config' in fname) and (fname.split('.')[-1] == 'json'):
        model_config_path = os.path.join(base_model_path, fname)
    elif ('ckpt' in fname) and (fname.split('.')[-1] == 'pt'):
        model_ckpt_path = os.path.join(base_model_path, fname)
    else:
        print(fname, 'does not exist')
print(model_config_path, model_ckpt_path)

# load ckpt and json
with open(model_config_path, 'r') as f:
    model_config = json.load(f)


### Read data

In [ ]:
# ----------------------------------
# Reading input/output data
# ----------------------------------
dataset = model_config['Data']['dataset']  
theta_type = model_config['Data']['theta_type']  
inc_neutral = model_config['Data']['include_neutral']  
use_facealigner = True if model_config['Data']['input_data_version'] == 'facealigner' else False

data_path = './datasets/Rafd_Aligned/Rafd_LANDMARKS'  # set data path
if dataset == 'Rafd':
    # dirty hack only used to get Rafd speaker ids, not continuously ordered
    data_csv_path = '/home/mlpboon/Downloads/Rafd/Rafd.csv'

print('Reading data')
if use_facealigner:
    if dataset == 'KDEF':
        from datasets.datasets import kdef_landmarks_facealigner
        x_train, y_train, x_test, y_test, train_list, test_list = \
            kdef_landmarks_facealigner(data_path, inc_neutral=inc_neutral)
    elif dataset == 'Rafd':
        from datasets.datasets import rafd_landmarks_facealigner
        x_train, y_train, x_test, y_test, train_list, test_list = \
            rafd_landmarks_facealigner(data_path, data_csv_path, inc_neutral=inc_neutral)
else:
    from datasets.datasets import import_kdef_landmark_synthesis
    x_train, y_train, x_test, y_test = import_kdef_landmark_synthesis(dtype=input_data_version)

n = x_train.shape[0]
m = y_train.shape[1]
nf = y_train.shape[2]
print('data dimensions', n, m, nf)

In [ ]:
# set ITL model
assert model_config['Kernels']['kernel_input_learnable'] == False
kernel_input = kernel.Gaussian(model_config['Kernels']['gamma_inp'])
kernel_output = kernel.Gaussian(model_config['Kernels']['gamma_out'])
kernel_freq = np.eye(nf) # can be added to ckpt or manually set as np.load(kernel_file)

# define emotion sampler - this can also be added to ckpt
if model_config['Data']['theta_type'] == 'aff':
    from datasets.datasets import import_affectnet_va_embedding
    affect_net_csv_path = './utils/landmark_utils/validation.csv'  # to be set if theta_type == 'aff'
    aff_emo_dict = import_affectnet_va_embedding(affect_net_csv_path)

    sampler_ = sampler.CircularSampler(data=dataset+theta_type,
                                       inc_neutral=inc_neutral,
                                       sample_dict=aff_emo_dict)
elif theta_type == '':
    sampler_ = sampler.CircularSampler(data=dataset,
                                       inc_neutral=inc_neutral)
sampler_.m = m

itl_model = model.SpeechSynthesisKernelModel(kernel_input, kernel_output,
                                             kernel_freq=torch.from_numpy(kernel_freq).float())

### Load model and predict

In [ ]:
ckpt = torch.load(model_ckpt_path)
itl_model.test_mode(x_train=x_train, thetas=sampler_.sample(m), alpha=ckpt['itl_alpha'])
pred_test = itl_model.forward(x_test, sampler_.sample(m))

In [ ]:
check_output = pred_test*128
check_output[0,0].reshape(68,2)

In [ ]:
%matplotlib inline
plt_x = x_test[0].numpy().reshape(68, 2)
plt_xt = pred_test[1, 4].detach().numpy().reshape(68, 2)
if use_facealigner:
    plt_x = plt_x * 128
    plt_xt = plt_xt * 128
plt_uv = plt_xt - plt_x
plt.quiver(plt_x[:, 0], plt_x[:, 1], plt_uv[:, 0], plt_uv[:, 1], angles='xy')
ax = plt.gca()
ax.invert_yaxis()
plt.show()